In [19]:
import pickle
import pandas as pd
import numpy as np

In [20]:
import src.process as process
import src.explore as explore
import src.model as model

In [21]:
start_yr = 2016
end_yr = 2020
path_blueprint = r'raw\data_{}.xlsx'
raw_data = process.merge_import_from_xlsx(start_yr, end_yr, path_blueprint)

data = process.select_clean_data(raw_data, min_instances = 50)

train_data, test_data = process.stratified_train_test_split(data)

In [22]:
lookup_table = pickle.load(open(r"assets/lookup.pkl", "rb" ))

In [23]:
%load_ext tensorboard
from tensorboard.plugins.hparams import api as hp
import tensorflow as tf
import tensorflow_addons as tfa
from sklearn.metrics import f1_score, accuracy_score, recall_score

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [24]:
HP_NUM_UNITS = hp.HParam('num_units', hp.Discrete([140, 150, 160, 200, 250, 300, 500]))
HP_DROPOUT = hp.HParam('dropout', hp.Discrete([0.3]))
HP_REC_DROPOUT = hp.HParam('rec_dropout', hp.Discrete([0.0]))
HP_ACTIVATION = hp.HParam('activation', hp.Discrete(['relu', 'softmax']))
HP_OPTIMIZER = hp.HParam('optimizer', hp.Discrete(['adam']))

METRIC_ACCURACY = 'accuracy'
METRIC_F1_SCORE = 'f1_score'
METRIC_RECALL = 'recall'

with tf.summary.create_file_writer('logs/hyperopt').as_default():
    hp.hparams_config(
    hparams=[HP_NUM_UNITS, HP_DROPOUT, HP_REC_DROPOUT, HP_ACTIVATION, HP_OPTIMIZER],
    metrics=[hp.Metric(METRIC_ACCURACY, display_name='Accuracy'), hp.Metric(METRIC_F1_SCORE, display_name='F1Score'), hp.Metric(METRIC_RECALL, display_name='Recall')])

In [29]:
def tune_lstm_model(hparams, data, lookup_table):
    
    #Input Data
    lstm_model_data = process.create_lstm_model_data(data, lookup_table)
    train_set, test_set = process.stratified_train_test_split(lstm_model_data, test_ratio = .2, random_state = 0)
    
    input_tensor  = tf.constant(train_set['padded_sentence'].tolist())
    labels = tf.constant(pd.get_dummies(train_set['Politikbereich']))
    BATCH_SIZE = 1000
    BUFFER_SIZE = labels.shape[0]
    dataset = tf.data.Dataset.from_tensor_slices((input_tensor, labels))
    dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)
    
    input_tensor_test = tf.constant(test_set['padded_sentence'].tolist())
    labels_test = tf.constant(pd.get_dummies(test_set['Politikbereich']))
    BATCH_SIZE = 1000
    BUFFER_SIZE = labels_test.shape[0]
    dataset_test = tf.data.Dataset.from_tensor_slices((input_tensor_test, labels_test))
    dataset_test = dataset_test.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)
    
    #Model Definition
    weights = np.array(lookup_table['vector'].tolist())
    num_classes = train_set['Politikbereich'].unique().size
    
    model = tf.keras.models.Sequential()
    model.add(tf.keras.layers.Embedding(input_dim=weights.shape[0],
              output_dim=weights.shape[1],
              embeddings_initializer=tf.keras.initializers.Constant(weights),
              trainable=False,
              mask_zero=True))
    model.add(tf.keras.layers.Masking(mask_value=0))
    model.add(tf.keras.layers.LSTM(hparams[HP_NUM_UNITS], return_sequences=False, 
               dropout=hparams[HP_DROPOUT], recurrent_dropout=hparams[HP_REC_DROPOUT]))
    model.add(tf.keras.layers.Dense(num_classes, activation=hparams[HP_ACTIVATION]))
    
    if hparams[HP_ACTIVATION] == 'softmax':
        model.compile(optimizer=hparams[HP_OPTIMIZER], loss=tf.keras.losses.CategoricalCrossentropy(from_logits=False), metrics=[tf.keras.metrics.CategoricalAccuracy(), tfa.metrics.F1Score(num_classes = num_classes, average = 'macro')])
    else:
        model.compile(optimizer=hparams[HP_OPTIMIZER], loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True), metrics=[tf.keras.metrics.CategoricalAccuracy(), tfa.metrics.F1Score(num_classes = num_classes, average = 'macro')])
    
    
    #Model Training
    callbacks = [tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10)]
                #tf.keras.callbacks.TensorBoard('logs/hparam_tuning/'),  # log metrics
                #hp.KerasCallback('logs/hparam_tuning/', hparams)]
    model.fit(dataset, validation_data=dataset_test, callbacks = callbacks, epochs=1000)
    
    #Model Evaluation
    #cross_loss, accuracy, macro_f1score = model.evaluate(dataset_test)
    
    prediction = model.predict(input_tensor_test, batch_size=None, verbose=0, steps=None, callbacks=None, max_queue_size=10, workers=1, use_multiprocessing=False)
    classes = pd.get_dummies(train_set['Politikbereich']).columns
    y_test = test_set['Politikbereich']
    y_pred = [classes[x] for x in prediction.argmax(axis=1)]
    
    accuracy = accuracy_score(y_test, y_pred)
    macro_f1score = f1_score(y_test, y_pred, average = 'macro')
    macro_recall = recall_score(y_test, y_pred, average = 'macro')
    

    return accuracy, macro_f1score, macro_recall


In [26]:
def run(run_dir, hparams, data, lookup_table, step):
    with tf.summary.create_file_writer(run_dir).as_default():
        hp.hparams(hparams)  # record the values used in this trial
        accuracy, macro_f1score, macro_recall = tune_lstm_model(hparams, train_data, lookup_table)
        tf.summary.scalar(name = METRIC_ACCURACY, data = accuracy, step= step)
        tf.summary.scalar(name = METRIC_F1_SCORE, data = macro_f1score, step = step)
        tf.summary.scalar(name = METRIC_RECALL, data = macro_recall, step = step)


In [30]:
#remove previous logs if wanted
#import shutil
#shutil.rmtree('logs/hparam_tuning')

session_num = 0

for num_units in HP_NUM_UNITS.domain.values:
    #for dropout_rate in (HP_DROPOUT.domain.min_value, HP_DROPOUT.domain.max_value):
    for dropout in HP_DROPOUT.domain.values:
        for rec_dropout in HP_REC_DROPOUT.domain.values:
            for activation in HP_ACTIVATION.domain.values:
                for optimizer in HP_OPTIMIZER.domain.values:
                    hparams = {HP_NUM_UNITS: num_units,
                               HP_DROPOUT: dropout,
                               HP_REC_DROPOUT: rec_dropout,
                               HP_ACTIVATION: activation,
                               HP_OPTIMIZER: optimizer}
            
                    run_name = "run-%d" % session_num
                    print('--- Starting trial: %s' % run_name)
                    print({h.name: hparams[h] for h in hparams})
                    #tune_lstm_model(hparams, train_data, lookup_table)
                    run('logs/hyperopt/' + run_name, hparams, train_data, lookup_table, session_num)
                    session_num += 1


--- Starting trial: run-0
{'num_units': 140, 'dropout': 0.3, 'rec_dropout': 0.0, 'activation': 'relu', 'optimizer': 'adam'}
Epoch 1/1000
13/13 [==============================] - 13s 747ms/step - loss: 3.1514 - categorical_accuracy: 0.1182 - f1_score: 0.0175 - val_loss: 2.7727 - val_categorical_accuracy: 0.1447 - val_f1_score: 0.0097
Epoch 2/1000
13/13 [==============================] - 8s 641ms/step - loss: 2.7040 - categorical_accuracy: 0.2534 - f1_score: 0.0225 - val_loss: 2.6223 - val_categorical_accuracy: 0.2893 - val_f1_score: 0.0280
Epoch 3/1000
13/13 [==============================] - 8s 638ms/step - loss: 2.5005 - categorical_accuracy: 0.3307 - f1_score: 0.0400 - val_loss: 2.3799 - val_categorical_accuracy: 0.3617 - val_f1_score: 0.0476
Epoch 4/1000
13/13 [==============================] - 8s 632ms/step - loss: 2.2797 - categorical_accuracy: 0.3697 - f1_score: 0.0526 - val_loss: 2.1694 - val_categorical_accuracy: 0.3910 - val_f1_score: 0.0633
Epoch 5/1000
13/13 [===============

13/13 [==============================] - 9s 672ms/step - loss: 1.9070 - categorical_accuracy: 0.4558 - f1_score: 0.1345 - val_loss: 1.8420 - val_categorical_accuracy: 0.4820 - val_f1_score: 0.1439
Epoch 9/1000
13/13 [==============================] - 9s 677ms/step - loss: 1.8318 - categorical_accuracy: 0.4749 - f1_score: 0.1437 - val_loss: 1.7841 - val_categorical_accuracy: 0.4913 - val_f1_score: 0.1490
Epoch 10/1000
13/13 [==============================] - 9s 675ms/step - loss: 1.7712 - categorical_accuracy: 0.4888 - f1_score: 0.1556 - val_loss: 1.7261 - val_categorical_accuracy: 0.5043 - val_f1_score: 0.1767
Epoch 11/1000
13/13 [==============================] - 9s 674ms/step - loss: 1.7182 - categorical_accuracy: 0.5125 - f1_score: 0.1991 - val_loss: 1.6327 - val_categorical_accuracy: 0.5393 - val_f1_score: 0.2258
Epoch 12/1000
13/13 [==============================] - 9s 676ms/step - loss: 1.6535 - categorical_accuracy: 0.5311 - f1_score: 0.2242 - val_loss: 1.6023 - val_categorical_

Epoch 47/1000
13/13 [==============================] - 9s 710ms/step - loss: 1.0212 - categorical_accuracy: 0.7019 - f1_score: 0.5172 - val_loss: 1.2576 - val_categorical_accuracy: 0.6520 - val_f1_score: 0.4458
Epoch 48/1000
13/13 [==============================] - 10s 771ms/step - loss: 1.0182 - categorical_accuracy: 0.7021 - f1_score: 0.5122 - val_loss: 1.2763 - val_categorical_accuracy: 0.6477 - val_f1_score: 0.4299
Epoch 49/1000
13/13 [==============================] - 11s 851ms/step - loss: 1.0268 - categorical_accuracy: 0.6960 - f1_score: 0.5103 - val_loss: 1.2539 - val_categorical_accuracy: 0.6563 - val_f1_score: 0.4410
Epoch 50/1000
13/13 [==============================] - 11s 834ms/step - loss: 1.0105 - categorical_accuracy: 0.7041 - f1_score: 0.5226 - val_loss: 1.2660 - val_categorical_accuracy: 0.6570 - val_f1_score: 0.4533
Epoch 51/1000
13/13 [==============================] - 10s 771ms/step - loss: 0.9967 - categorical_accuracy: 0.7098 - f1_score: 0.5317 - val_loss: 1.2588

13/13 [==============================] - 10s 740ms/step - loss: 1.7767 - categorical_accuracy: 0.4875 - f1_score: 0.1516 - val_loss: 1.6870 - val_categorical_accuracy: 0.5137 - val_f1_score: 0.1707
Epoch 10/1000
13/13 [==============================] - 10s 785ms/step - loss: 1.7046 - categorical_accuracy: 0.5145 - f1_score: 0.1920 - val_loss: 1.6527 - val_categorical_accuracy: 0.5377 - val_f1_score: 0.2174
Epoch 11/1000
13/13 [==============================] - 10s 758ms/step - loss: 1.6492 - categorical_accuracy: 0.5315 - f1_score: 0.2220 - val_loss: 1.6036 - val_categorical_accuracy: 0.5400 - val_f1_score: 0.2249
Epoch 12/1000
13/13 [==============================] - 10s 785ms/step - loss: 1.6117 - categorical_accuracy: 0.5383 - f1_score: 0.2258 - val_loss: 1.5809 - val_categorical_accuracy: 0.5533 - val_f1_score: 0.2375
Epoch 13/1000
13/13 [==============================] - 10s 750ms/step - loss: 1.5685 - categorical_accuracy: 0.5510 - f1_score: 0.2419 - val_loss: 1.5339 - val_catego

Epoch 48/1000
13/13 [==============================] - 10s 758ms/step - loss: 1.0191 - categorical_accuracy: 0.6991 - f1_score: 0.5132 - val_loss: 1.2457 - val_categorical_accuracy: 0.6513 - val_f1_score: 0.4343
Epoch 49/1000
13/13 [==============================] - 11s 826ms/step - loss: 1.0105 - categorical_accuracy: 0.7036 - f1_score: 0.5149 - val_loss: 1.2604 - val_categorical_accuracy: 0.6587 - val_f1_score: 0.4464
Epoch 50/1000
13/13 [==============================] - 12s 894ms/step - loss: 0.9947 - categorical_accuracy: 0.7078 - f1_score: 0.5270 - val_loss: 1.2820 - val_categorical_accuracy: 0.6470 - val_f1_score: 0.4308
Epoch 51/1000
13/13 [==============================] - 10s 776ms/step - loss: 0.9932 - categorical_accuracy: 0.7062 - f1_score: 0.5385 - val_loss: 1.2555 - val_categorical_accuracy: 0.6553 - val_f1_score: 0.4595
Epoch 52/1000
13/13 [==============================] - 10s 811ms/step - loss: 0.9840 - categorical_accuracy: 0.7109 - f1_score: 0.5396 - val_loss: 1.263

...

Epoch 18/1000
13/13 [==============================] - 16s 1s/step - loss: 1.3281 - categorical_accuracy: 0.6178 - f1_score: 0.3727 - val_loss: 1.3727 - val_categorical_accuracy: 0.6067 - val_f1_score: 0.3570
Epoch 19/1000
13/13 [==============================] - 16s 1s/step - loss: 1.3088 - categorical_accuracy: 0.6217 - f1_score: 0.3688 - val_loss: 1.3766 - val_categorical_accuracy: 0.6047 - val_f1_score: 0.3383
Epoch 20/1000
13/13 [==============================] - 16s 1s/step - loss: 1.3025 - categorical_accuracy: 0.6201 - f1_score: 0.3808 - val_loss: 1.3668 - val_categorical_accuracy: 0.6010 - val_f1_score: 0.3442
Epoch 21/1000
13/13 [==============================] - 16s 1s/step - loss: 1.2718 - categorical_accuracy: 0.6300 - f1_score: 0.3848 - val_loss: 1.3592 - val_categorical_accuracy: 0.6080 - val_f1_score: 0.3441
Epoch 22/1000
13/13 [==============================] - 16s 1s/step - loss: 1.2576 - categorical_accuracy: 0.6348 - f1_score: 0.4083 - val_loss: 1.3319 - val_categor

13/13 [==============================] - 20s 2s/step - loss: 0.9990 - categorical_accuracy: 0.7042 - f1_score: 0.5388 - val_loss: 1.2429 - val_categorical_accuracy: 0.6613 - val_f1_score: 0.4342
Epoch 41/1000
13/13 [==============================] - 20s 2s/step - loss: 0.9715 - categorical_accuracy: 0.7108 - f1_score: 0.5422 - val_loss: 1.2429 - val_categorical_accuracy: 0.6577 - val_f1_score: 0.4470
Epoch 42/1000
13/13 [==============================] - 19s 2s/step - loss: 0.9520 - categorical_accuracy: 0.7189 - f1_score: 0.5577 - val_loss: 1.2645 - val_categorical_accuracy: 0.6560 - val_f1_score: 0.4591
Epoch 43/1000
13/13 [==============================] - 19s 1s/step - loss: 0.9524 - categorical_accuracy: 0.7170 - f1_score: 0.5624 - val_loss: 1.2531 - val_categorical_accuracy: 0.6580 - val_f1_score: 0.4573
Epoch 44/1000
13/13 [==============================] - 20s 2s/step - loss: 0.9353 - categorical_accuracy: 0.7240 - f1_score: 0.5709 - val_loss: 1.2398 - val_categorical_accuracy:

Epoch 25/1000
13/13 [==============================] - 20s 2s/step - loss: 1.2145 - categorical_accuracy: 0.6441 - f1_score: 0.4331 - val_loss: 1.3075 - val_categorical_accuracy: 0.6270 - val_f1_score: 0.4006
Epoch 26/1000
13/13 [==============================] - 20s 2s/step - loss: 1.2091 - categorical_accuracy: 0.6465 - f1_score: 0.4324 - val_loss: 1.3051 - val_categorical_accuracy: 0.6287 - val_f1_score: 0.3922
Epoch 27/1000
13/13 [==============================] - 20s 2s/step - loss: 1.1927 - categorical_accuracy: 0.6473 - f1_score: 0.4344 - val_loss: 1.2836 - val_categorical_accuracy: 0.6287 - val_f1_score: 0.4058
Epoch 28/1000
13/13 [==============================] - 20s 2s/step - loss: 1.1624 - categorical_accuracy: 0.6565 - f1_score: 0.4431 - val_loss: 1.2698 - val_categorical_accuracy: 0.6360 - val_f1_score: 0.4064
Epoch 29/1000
13/13 [==============================] - 20s 2s/step - loss: 1.1479 - categorical_accuracy: 0.6615 - f1_score: 0.4618 - val_loss: 1.2757 - val_categor

Epoch 14/1000
13/13 [==============================] - 42s 3s/step - loss: 1.4830 - categorical_accuracy: 0.5777 - f1_score: 0.2921 - val_loss: 1.5075 - val_categorical_accuracy: 0.5773 - val_f1_score: 0.3020
Epoch 15/1000
13/13 [==============================] - 42s 3s/step - loss: 1.4574 - categorical_accuracy: 0.5862 - f1_score: 0.3150 - val_loss: 1.4473 - val_categorical_accuracy: 0.5940 - val_f1_score: 0.3196
Epoch 16/1000
13/13 [==============================] - 42s 3s/step - loss: 1.4302 - categorical_accuracy: 0.5912 - f1_score: 0.3336 - val_loss: 1.4405 - val_categorical_accuracy: 0.5950 - val_f1_score: 0.3113
Epoch 17/1000
13/13 [==============================] - 42s 3s/step - loss: 1.3915 - categorical_accuracy: 0.6015 - f1_score: 0.3476 - val_loss: 1.4217 - val_categorical_accuracy: 0.5937 - val_f1_score: 0.3378
Epoch 18/1000
13/13 [==============================] - 42s 3s/step - loss: 1.3518 - categorical_accuracy: 0.6141 - f1_score: 0.3743 - val_loss: 1.4206 - val_categor

Epoch 33/1000
13/13 [==============================] - 44s 3s/step - loss: 1.0339 - categorical_accuracy: 0.6932 - f1_score: 0.5358 - val_loss: 1.2793 - val_categorical_accuracy: 0.6463 - val_f1_score: 0.4264
Epoch 34/1000
13/13 [==============================] - 43s 3s/step - loss: 1.0116 - categorical_accuracy: 0.7004 - f1_score: 0.5491 - val_loss: 1.2741 - val_categorical_accuracy: 0.6500 - val_f1_score: 0.4520
Epoch 35/1000
13/13 [==============================] - 42s 3s/step - loss: 1.0086 - categorical_accuracy: 0.6982 - f1_score: 0.5374 - val_loss: 1.2851 - val_categorical_accuracy: 0.6420 - val_f1_score: 0.4405
Epoch 36/1000
13/13 [==============================] - 42s 3s/step - loss: 0.9904 - categorical_accuracy: 0.7052 - f1_score: 0.5510 - val_loss: 1.2828 - val_categorical_accuracy: 0.6457 - val_f1_score: 0.4505
Epoch 37/1000
13/13 [==============================] - 44s 3s/step - loss: 0.9625 - categorical_accuracy: 0.7116 - f1_score: 0.5700 - val_loss: 1.2832 - val_categor

In [ ]:
%tensorboard --logdir logs/hyperopt